In [103]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData
import pandas as pd
import requests
import json
from pathlib import Path
import sqlite3

In [138]:
Path('my_data.db').touch()

In [147]:
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

In [148]:
# grab country_data from our api to merge with coordinate df
url = "http://127.0.0.1:5000/country_data"
r = requests.get(url).json()

In [149]:
df = pd.DataFrame(r).drop(columns = ['remote_ratio','salary_in_usd'])
df

,company_location
0,AE
1,AL
2,AM
3,AR
4,AS
...,...
67,TH
68,TR
69,UA
70,US


In [120]:
#read in csv from https://gist.github.com/tadast/8827699
coord_df = pd.read_csv(r"C:\Users\Stan\Downloads\countries_codes_and_coordinates.csv")
coord_df.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""


In [121]:
coord_df = coord_df[['Country','Alpha-2 code','Latitude (average)', 'Longitude (average)']]

In [122]:
coord_df["Alpha-2 code"] = coord_df["Alpha-2 code"].str.replace('"', '')
coord_df["Latitude (average)"] = coord_df["Latitude (average)"].str.replace('"', '')
coord_df["Longitude (average)"] = coord_df["Longitude (average)"].str.replace('"', '')
coord_df["Alpha-2 code"] = coord_df["Alpha-2 code"].str.replace(' ', '')
coord_df["Latitude (average)"] = coord_df["Latitude (average)"].str.replace(' ', '')
coord_df["Longitude (average)"] = coord_df["Longitude (average)"].str.replace(' ', '')

In [123]:
coord_df = coord_df.rename(columns = {"Alpha-2 code":"company_location"})
coord_df

,Country,company_location,Latitude (average),Longitude (average)
0,Afghanistan,AF,33,65
1,Albania,AL,41,20
2,Algeria,DZ,28,3
3,American Samoa,AS,-14.3333,-170
4,Andorra,AD,42.5,1.6
...,...,...,...,...
251,Wallis and Futuna,WF,-13.3,-176.2
252,Western Sahara,EH,24.5,-13
253,Yemen,YE,15,48
254,Zambia,ZM,-15,30


In [124]:
merged_df = pd.merge(df, coord_df, on = "company_location", how = 'inner')
merged_df

,company_location,Country,Latitude (average),Longitude (average)
0,AE,United Arab Emirates,24,54
1,AL,Albania,41,20
2,AM,Armenia,40,45
3,AR,Argentina,-34,-64
4,AS,American Samoa,-14.3333,-170
...,...,...,...,...
70,TR,Turkey,39,35
71,UA,Ukraine,49,32
72,US,United States,38,-97
73,VN,Viet Nam,16,106


In [125]:
merged_df[merged_df.duplicated(subset = ['company_location', 'Latitude (average)', 'Longitude (average)'])]

,company_location,Country,Latitude (average),Longitude (average)
10,BO,Bolivia,-17,-65
64,RU,Russia,60,100
74,VN,Vietnam,16,106


In [126]:
df_cleaned = merged_df.drop([merged_df.index[10], merged_df.index[64], merged_df.index[74]])
df_cleaned

,company_location,Country,Latitude (average),Longitude (average)
0,AE,United Arab Emirates,24,54
1,AL,Albania,41,20
2,AM,Armenia,40,45
3,AR,Argentina,-34,-64
4,AS,American Samoa,-14.3333,-170
...,...,...,...,...
69,TH,Thailand,15,100
70,TR,Turkey,39,35
71,UA,Ukraine,49,32
72,US,United States,38,-97


In [127]:
df_cleaned = df_cleaned.astype({'Latitude (average)':'float64','Longitude (average)':'float64'})

In [128]:
df_cleaned = df_cleaned.rename(columns = {'company_location':'Country_Code','Latitude (average)':'Latitude','Longitude (average)':'Longitude'})
df_cleaned = df_cleaned[['Country', 'Country_Code','Latitude', 'Longitude']]
df_cleaned.head()

,Country,Country_Code,Latitude,Longitude
0,United Arab Emirates,AE,24.0000,54.0
1,Albania,AL,41.0000,20.0
2,Armenia,AM,40.0000,45.0
3,Argentina,AR,-34.0000,-64.0
4,American Samoa,AS,-14.3333,-170.0


In [129]:
df_cleaned.to_csv("Data/country_locations.csv")

### Load csv into database

In [140]:
c.execute('''CREATE TABLE country_locations (
    Country TEXT,
    Country_Code TEXT PRIMARY KEY,
    Latitude DECIMAL,
    Longitude DECIMAL
    );
''')

In [141]:
# load the data into a Pandas DataFrame
data = pd.read_csv(r"Data\country_locations.csv").drop(columns = "Unnamed: 0")
# write the data to a sqlite table
data.to_sql('country_locations', conn, if_exists='append', index = False)

72

In [142]:
data = c.execute('''SELECT * FROM country_locations''').fetchall()
data

[('United Arab Emirates', 'AE', 24, 54),
 ('Albania', 'AL', 41, 20),
 ('Armenia', 'AM', 40, 45),
 ('Argentina', 'AR', -34, -64),
 ('American Samoa', 'AS', -14.3333, -170),
 ('Austria', 'AT', 47.3333, 13.3333),
 ('Australia', 'AU', -27, 133),
 ('Bosnia and Herzegovina', 'BA', 44, 18),
 ('Belgium', 'BE', 50.8333, 4),
 ('Bolivia, Plurinational State of', 'BO', -17, -65),
 ('Brazil', 'BR', -10, -55),
 ('Bahamas', 'BS', 24.25, -76),
 ('Canada', 'CA', 60, -95),
 ('Central African Republic', 'CF', 7, 21),
 ('Switzerland', 'CH', 47, 8),
 ('Chile', 'CL', -30, -71),
 ('China', 'CN', 35, 105),
 ('Colombia', 'CO', 4, -72),
 ('Costa Rica', 'CR', 10, -84),
 ('Czech Republic', 'CZ', 49.75, 15.5),
 ('Germany', 'DE', 51, 9),
 ('Denmark', 'DK', 56, 10),
 ('Algeria', 'DZ', 28, 3),
 ('Estonia', 'EE', 59, 26),
 ('Egypt', 'EG', 27, 30),
 ('Spain', 'ES', 40, -4),
 ('Finland', 'FI', 64, 26),
 ('France', 'FR', 46, 2),
 ('United Kingdom', 'GB', 54, -2),
 ('Ghana', 'GH', 8, -2),
 ('Greece', 'GR', 39, 22),
 ('Hon

### create main table in database and load csv

In [143]:
# new cell block
# different from one below because i adds a id_card col as primary key
# this allows the db file to work with sqlalchemy 
c.execute('''CREATE TABLE data_science (
    work_year INTEGER
    experience_level TEXT,
    employment_type TEXT,
    job_title TEXT,
    salary INTEGER,
    salary_currency TEXT,
    salary_in_usd INTEGER,
    employee_residence TEXT,
    remote_ratio INTEGER,
    company_location TEXT,
    company_size TEXT,
    id_card Serial PRIMARY KEY,
    FOREIGN KEY (company_location) REFERENCES country_locations(Country_Code)
);
''')

In [144]:
# load the data into a Pandas DataFrame
data = pd.read_csv(r"Data\ds_salaries.csv")
# write the data to a sqlite table
data.to_sql('data_salaries', conn, if_exists='append', index = False)

3755

In [145]:
data = c.execute('''SELECT * FROM data_salaries''').fetchall()
data

[(2023,
  'SE',
  'FT',
  'Principal Data Scientist',
  80000,
  'EUR',
  85847,
  'ES',
  100,
  'ES',
  'L'),
 (2023, 'MI', 'CT', 'ML Engineer', 30000, 'USD', 30000, 'US', 100, 'US', 'S'),
 (2023, 'MI', 'CT', 'ML Engineer', 25500, 'USD', 25500, 'US', 100, 'US', 'S'),
 (2023,
  'SE',
  'FT',
  'Data Scientist',
  175000,
  'USD',
  175000,
  'CA',
  100,
  'CA',
  'M'),
 (2023,
  'SE',
  'FT',
  'Data Scientist',
  120000,
  'USD',
  120000,
  'CA',
  100,
  'CA',
  'M'),
 (2023,
  'SE',
  'FT',
  'Applied Scientist',
  222200,
  'USD',
  222200,
  'US',
  0,
  'US',
  'L'),
 (2023,
  'SE',
  'FT',
  'Applied Scientist',
  136000,
  'USD',
  136000,
  'US',
  0,
  'US',
  'L'),
 (2023,
  'SE',
  'FT',
  'Data Scientist',
  219000,
  'USD',
  219000,
  'CA',
  0,
  'CA',
  'M'),
 (2023,
  'SE',
  'FT',
  'Data Scientist',
  141000,
  'USD',
  141000,
  'CA',
  0,
  'CA',
  'M'),
 (2023,
  'SE',
  'FT',
  'Data Scientist',
  147100,
  'USD',
  147100,
  'US',
  0,
  'US',
  'M'),
 (2023

In [146]:
conn.close()